In [1]:
from poloniex import Poloniex
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from bokeh.palettes import Category10
import pandas as pd
import time
output_notebook()

TOOLS="hover,crosshair,pan,wheel_zoom,zoom_in,zoom_out,box_zoom,undo,redo,reset,tap,save,box_select,poly_select,lasso_select,"

# Pandas config
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

#Definição dos mercados de AltCoins
markets = ["USDT_BTC", "USDT_DASH", "USDT_LTC", "USDT_NXT", "USDT_STR", "USDT_XMR", "USDT_XRP", "USDT_ETH"]

#Definição do Periodo da Análise
start_date = 1516009602
end_date = int(time.time())

# Intervalo dos candles de 300 segundos 
period = 300 

df_markets = {}

poloniex = Poloniex()

Loading BokehJS ...

In [2]:
def add_diff(market, period, start_date, end_date):
    df_markets[market] = pd.DataFrame(poloniex.returnChartData(market, period, int(start_date), int(end_date)))
    first_value = df_markets[market].head(1)["weightedAverage"][0]
    df_markets[market]["change"] = df_markets[market].apply(get_diff, first_value=(first_value), axis=1)
    
def get_diff(df_markets, first_value):   
    #Adicionando % de diferença entre o valor inicial e o ticker atual
    percentual_diff = ((float(df_markets.weightedAverage) * 100) / float(first_value)) -100 
    return percentual_diff

In [3]:
#Carregando Datasets
for market in markets:
    add_diff(market, period, start_date, end_date)  

In [4]:
#Carrega Gráfico
p = figure(title="Dependência", x_axis_label='date', y_axis_label='change', plot_width=950, plot_height=720, tools=TOOLS)
index = 0

#Definição de limites superior e inferior do mercado Bitcoin
p.rect(df_markets["USDT_BTC"].date, df_markets["USDT_BTC"].change, line_color=Category10[10][2] ,legend="Bitcoin", line_alpha=0.3, width=20, height=10)
p.rect(df_markets["USDT_BTC"].date, df_markets["USDT_BTC"].change, line_color=Category10[10][3] ,legend="Bitcoin", line_alpha=0.3, width=3, height=3)
# -- 

# Carregando informações dos mercados de AltCoins
for market in markets:
    p.line(df_markets[market].date, df_markets[market].change, line_color=Category10[10][index] ,legend=market, line_width=2)
    index += 1

p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)